In [2]:
import pandas as pd

# Read in the train data and get image information
train_df = pd.read_csv('../data/train.csv')
train_df['worm_type'] = pd.Categorical(train_df['worm_type'])
train_df['worm_code'] = train_df['worm_type'].cat.codes + 1
train_df

,image_id_worm,worm_type,number_of_worms,worm_code
0,id_0002ea6f15c7fa6f4c221783.jpg,pbw,51,2
1,id_0005ef295aafe6acc63587db.jpg,pbw,8,2
2,id_00084298dd030a500033ff78.jpg,NaN,0,0
3,id_00093f2c76f6488737325859.jpg,pbw,12,2
4,id_000b2e6c437c643f25d4a6c3.jpg,pbw,87,2
...,...,...,...,...
9895,id_ffe6e193dd78b7258d864f6b.jpg,abw,5,1
9896,id_ffe98feb1e592e887b81c3f0.jpg,pbw,95,2
9897,id_ffef90ecf47baa38434f84fb.jpg,pbw,26,2
9898,id_fff06bb2e59bfce437ac9fc0.jpg,NaN,0,0


In [3]:
# Remove duplicates from the training dataframe
unique_train_df = train_df.drop_duplicates(subset=['image_id_worm'])
unique_train_df

,image_id_worm,worm_type,number_of_worms,worm_code
0,id_0002ea6f15c7fa6f4c221783.jpg,pbw,51,2
1,id_0005ef295aafe6acc63587db.jpg,pbw,8,2
2,id_00084298dd030a500033ff78.jpg,NaN,0,0
3,id_00093f2c76f6488737325859.jpg,pbw,12,2
4,id_000b2e6c437c643f25d4a6c3.jpg,pbw,87,2
...,...,...,...,...
9895,id_ffe6e193dd78b7258d864f6b.jpg,abw,5,1
9896,id_ffe98feb1e592e887b81c3f0.jpg,pbw,95,2
9897,id_ffef90ecf47baa38434f84fb.jpg,pbw,26,2
9898,id_fff06bb2e59bfce437ac9fc0.jpg,NaN,0,0


In [4]:
# Split train and test
from sklearn.model_selection import train_test_split

train, validation = train_test_split(unique_train_df, test_size=0.1, random_state=1)

In [10]:
import os
import shutil

def generate_dataset_folders(df, dataset_type):
    print('Generating', dataset_type, 'dataset')

    # Remove/create output directory
    dataset_path = f'../data/regression/{dataset_type}/'
    images_path = '../data/images/'
    if os.path.exists(dataset_path):
        shutil.rmtree(dataset_path)
    os.mkdir(dataset_path)
    
    for _, row in df.iterrows():
        img_name = row['image_id_worm']
        img_prefix = img_name.split('.')[0]

        shutil.copy(f'{images_path}{img_name}', f'{dataset_path}{img_name}')
        
        if 'worm_code' in df.columns:
            if row['worm_code'] == -1:
                # Create an empty txt file
                open(f'{dataset_path}{img_prefix}.txt', 'a').close()
            else:
                # Retrieve the bounding boxes in the yolo format
                boxes_df = train_df[train_df['image_id_worm'] == f'{img_name}']
                boxes_df = boxes_df.drop(columns=['image_id_worm', 'worm_code'])
                boxes_df.to_csv(f'{dataset_path}{img_prefix}.txt', sep ='\t', header=False, index=False)

In [11]:
generate_dataset_folders(train, 'training')

Generating training dataset


In [12]:
generate_dataset_folders(validation, 'validation')

Generating validation dataset


In [13]:
validation.to_csv('../data/image_regression/validation_set.csv', index=False)

OSError: Cannot save file into a non-existent directory: '../data/image_regression'

In [ ]:
test_df = pd.read_csv('../data/test.csv')
generate_dataset_folders(test_df, 'testing')